In [1]:
from imutils import paths
import cv2
import os
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

import torch.autograd as autograd

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import torch
device = torch.device("cuda")

In [22]:
image_paths = list(paths.list_images('data/Caltech101/001'))
#image_paths = list(paths.list_images('data/cars_side-view'))

In [23]:
data = []
labels = []
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    data.append(img)
    labels.append(label)
    if len(labels) > 5000:
        break
    
data = np.array(data)
labels = np.array(labels)

100%|██████████| 800/800 [00:00<00:00, 1133.62it/s]
<ipython-input-23-b8270de6310a>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


In [24]:

lb = LabelEncoder()
labels = lb.fit_transform(labels)
print(f"Total Number of Classes: {len(lb.classes_)}")

Total Number of Classes: 1


In [7]:
Counter(labels)

Counter({0: 123})

In [25]:
from sklearn.model_selection import train_test_split
# divide the data into train and test set
(x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.1, stratify=labels, random_state=42)
print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}")

x_train examples: (720,)
x_test examples: (80,)


In [26]:
dataset_config = {'size': 64, 'channels': 3, 'classes': 1}

In [27]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((dataset_config['size'], dataset_config['size'])),
    transforms.ToTensor(),
    #transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((dataset_config['size'],dataset_config['size'])),
    transforms.ToTensor(),
    #transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])    

In [11]:
BS = 32
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BS, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.ToTensor()),
    batch_size=BS, shuffle=True)


Files already downloaded and verified


In [28]:
BS = 32
# custom dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels= None, transforms = None):
        self.labels = labels
        self.images = images
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        data = self.images[index][:]
        
        if self.transforms:
            data = self.transforms(data)
            
        
        return (data, self.labels[index])
        
train_data = CustomDataset(x_train, y_train, train_transforms)
test_data = CustomDataset(x_test, y_test, val_transform)       

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=BS, shuffle=True, num_workers=4, drop_last=False) 

### --- Main --

In [29]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable

from torchvision.utils import save_image

def gradients(y, x):
    return autograd.grad(
                outputs=y, inputs=x, retain_graph=True,
                create_graph=True, grad_outputs=torch.ones_like(y), only_inputs=True)[0]

In [30]:
class VAE_Cifar10(nn.Module):
    def __init__(self, label = 'cifar10', image_size = dataset_config['size'], channel_num = dataset_config['channels'], kernel_num = 128, z_size=128):
        # configurations
        super().__init__()
        self.label = label
        self.image_size = image_size
        self.channel_num = channel_num
        self.kernel_num = kernel_num
        self.z_size = z_size

        # encoder
        self.encoder = nn.Sequential(
            self._conv(channel_num, kernel_num // 4),
            self._conv(kernel_num // 4, kernel_num // 2),
            self._conv(kernel_num // 2, kernel_num),
        )

        # encoded feature's size and volume

        # q
        self.q_mean = self._linear(self.feature_volume, z_size, relu=False)
        self.q_logvar = self._linear(self.feature_volume, z_size, relu=False)

        # projection
        self.project = self._linear(z_size, self.feature_volume, relu=False)

        # decoder
        self.decoder = nn.Sequential(
            self._deconv(kernel_num, kernel_num // 2),
            self._deconv(kernel_num // 2, kernel_num // 4),
            self._deconv(kernel_num // 4, channel_num),
            nn.Sigmoid()
        )


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std


    def forward(self, x):
        encoded = self.encoder(x)

        # sample latent code z from q given x.
        mean, logvar = self.q(encoded)
        z = self.z(mean, logvar)
        z_projected = self.project(z).view(
            -1, self.kernel_num,
            self.feature_size,
            self.feature_size,
        )

        # reconstruct x from z
        x_reconstructed = self.decoder(z_projected)

        return x_reconstructed, mean, logvar
    
    def q(self, encoded):
        unrolled = encoded.view(-1, self.feature_volume)
        return self.q_mean(unrolled), self.q_logvar(unrolled)

    def z(self, mean, logvar):
        std = logvar.mul(0.5).exp_()
        eps = (Variable(torch.randn(std.size())).to(device))
        
        return mean#eps.mul(std).add_(mean)
    
    
    # ======
    # Layers
    # ======

    def _conv(self, channel_size, kernel_num):
        return nn.Sequential(
            nn.Conv2d(
                channel_size, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.InstanceNorm2d(kernel_num),
            nn.ReLU(),
        )

    def _deconv(self, channel_num, kernel_num):
        return nn.Sequential(
            nn.ConvTranspose2d(
                channel_num, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.InstanceNorm2d(kernel_num),
            nn.ReLU(),
        )

    def _linear(self, in_size, out_size, relu=True):
        return nn.Sequential(
            nn.Linear(in_size, out_size),
            nn.ReLU(),
        ) if relu else nn.Linear(in_size, out_size)

model = VAE_Cifar10().to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [32]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    MSE = (recon_x - x.view(-1,  dataset_config['channels'],dataset_config['size'],dataset_config['size'])) ** 2
    MSE = MSE.sum(dim=(-1,-2,-3))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=-1)

    return MSE + 0.3 * KLD

In [33]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar).mean(dim=0)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [34]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).mean(dim=0).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch[:n].view(n,  dataset_config['channels'], dataset_config['size'], dataset_config['size'])[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [35]:
for epoch in range(1, 100 + 1):
        train(epoch)
        test(epoch)

Train Epoch: 1 [0/720 (0%)]	Loss: 34.416878
Train Epoch: 1 [64/720 (9%)]	Loss: 32.253586
Train Epoch: 1 [128/720 (17%)]	Loss: 28.188232
Train Epoch: 1 [192/720 (26%)]	Loss: 28.446091
Train Epoch: 1 [256/720 (35%)]	Loss: 27.629402
Train Epoch: 1 [320/720 (43%)]	Loss: 25.524319
Train Epoch: 1 [384/720 (52%)]	Loss: 26.149231
Train Epoch: 1 [448/720 (61%)]	Loss: 25.002098
Train Epoch: 1 [512/720 (70%)]	Loss: 20.960619
Train Epoch: 1 [576/720 (78%)]	Loss: 25.569416
Train Epoch: 1 [640/720 (87%)]	Loss: 25.051151
Train Epoch: 1 [352/720 (96%)]	Loss: 25.144415
====> Epoch: 1 Average loss: 26.3103
====> Test set loss: 23.0203
Train Epoch: 2 [0/720 (0%)]	Loss: 19.983946
Train Epoch: 2 [64/720 (9%)]	Loss: 24.922157
Train Epoch: 2 [128/720 (17%)]	Loss: 18.742249
Train Epoch: 2 [192/720 (26%)]	Loss: 22.263584
Train Epoch: 2 [256/720 (35%)]	Loss: 21.157627
Train Epoch: 2 [320/720 (43%)]	Loss: 19.543890
Train Epoch: 2 [384/720 (52%)]	Loss: 17.565090
Train Epoch: 2 [448/720 (61%)]	Loss: 19.897469
Trai

Train Epoch: 14 [448/720 (61%)]	Loss: 14.935298
Train Epoch: 14 [512/720 (70%)]	Loss: 12.638957
Train Epoch: 14 [576/720 (78%)]	Loss: 14.630822
Train Epoch: 14 [640/720 (87%)]	Loss: 16.816351
Train Epoch: 14 [352/720 (96%)]	Loss: 11.630459
====> Epoch: 14 Average loss: 13.5296
====> Test set loss: 15.7205
Train Epoch: 15 [0/720 (0%)]	Loss: 14.008562
Train Epoch: 15 [64/720 (9%)]	Loss: 12.129624
Train Epoch: 15 [128/720 (17%)]	Loss: 13.126711
Train Epoch: 15 [192/720 (26%)]	Loss: 14.273884
Train Epoch: 15 [256/720 (35%)]	Loss: 12.579286
Train Epoch: 15 [320/720 (43%)]	Loss: 12.902699
Train Epoch: 15 [384/720 (52%)]	Loss: 12.563902
Train Epoch: 15 [448/720 (61%)]	Loss: 13.191581
Train Epoch: 15 [512/720 (70%)]	Loss: 15.257868
Train Epoch: 15 [576/720 (78%)]	Loss: 14.263505
Train Epoch: 15 [640/720 (87%)]	Loss: 13.837521
Train Epoch: 15 [352/720 (96%)]	Loss: 13.090438
====> Epoch: 15 Average loss: 13.3835
====> Test set loss: 15.3525
Train Epoch: 16 [0/720 (0%)]	Loss: 11.720374
Train Epoc

Train Epoch: 27 [512/720 (70%)]	Loss: 14.602056
Train Epoch: 27 [576/720 (78%)]	Loss: 13.249741
Train Epoch: 27 [640/720 (87%)]	Loss: 11.125199
Train Epoch: 27 [352/720 (96%)]	Loss: 12.441035
====> Epoch: 27 Average loss: 12.7973
====> Test set loss: 15.2534
Train Epoch: 28 [0/720 (0%)]	Loss: 12.829718
Train Epoch: 28 [64/720 (9%)]	Loss: 14.099239
Train Epoch: 28 [128/720 (17%)]	Loss: 13.832338
Train Epoch: 28 [192/720 (26%)]	Loss: 15.723982
Train Epoch: 28 [256/720 (35%)]	Loss: 12.216627
Train Epoch: 28 [320/720 (43%)]	Loss: 12.993664
Train Epoch: 28 [384/720 (52%)]	Loss: 13.799588
Train Epoch: 28 [448/720 (61%)]	Loss: 12.187080
Train Epoch: 28 [512/720 (70%)]	Loss: 10.316810
Train Epoch: 28 [576/720 (78%)]	Loss: 12.900422
Train Epoch: 28 [640/720 (87%)]	Loss: 12.756889
Train Epoch: 28 [352/720 (96%)]	Loss: 14.312614
====> Epoch: 28 Average loss: 12.7921
====> Test set loss: 15.0995
Train Epoch: 29 [0/720 (0%)]	Loss: 12.204105
Train Epoch: 29 [64/720 (9%)]	Loss: 12.799250
Train Epoch:

Train Epoch: 40 [448/720 (61%)]	Loss: 12.141940
Train Epoch: 40 [512/720 (70%)]	Loss: 13.714654
Train Epoch: 40 [576/720 (78%)]	Loss: 12.038718
Train Epoch: 40 [640/720 (87%)]	Loss: 11.700603
Train Epoch: 40 [352/720 (96%)]	Loss: 14.545260
====> Epoch: 40 Average loss: 12.4819
====> Test set loss: 14.8220
Train Epoch: 41 [0/720 (0%)]	Loss: 12.335330
Train Epoch: 41 [64/720 (9%)]	Loss: 12.206857
Train Epoch: 41 [128/720 (17%)]	Loss: 12.513545
Train Epoch: 41 [192/720 (26%)]	Loss: 11.938486
Train Epoch: 41 [256/720 (35%)]	Loss: 11.161780
Train Epoch: 41 [320/720 (43%)]	Loss: 11.716285
Train Epoch: 41 [384/720 (52%)]	Loss: 12.217349
Train Epoch: 41 [448/720 (61%)]	Loss: 11.654828
Train Epoch: 41 [512/720 (70%)]	Loss: 13.092831
Train Epoch: 41 [576/720 (78%)]	Loss: 14.985875
Train Epoch: 41 [640/720 (87%)]	Loss: 12.829641
Train Epoch: 41 [352/720 (96%)]	Loss: 10.644513
====> Epoch: 41 Average loss: 12.4523
====> Test set loss: 14.8164
Train Epoch: 42 [0/720 (0%)]	Loss: 11.577024
Train Epoc

Train Epoch: 53 [448/720 (61%)]	Loss: 13.255745
Train Epoch: 53 [512/720 (70%)]	Loss: 9.944128
Train Epoch: 53 [576/720 (78%)]	Loss: 11.566618
Train Epoch: 53 [640/720 (87%)]	Loss: 11.471787
Train Epoch: 53 [352/720 (96%)]	Loss: 13.386800
====> Epoch: 53 Average loss: 12.3853
====> Test set loss: 14.9198
Train Epoch: 54 [0/720 (0%)]	Loss: 13.162567
Train Epoch: 54 [64/720 (9%)]	Loss: 12.101654
Train Epoch: 54 [128/720 (17%)]	Loss: 11.398981
Train Epoch: 54 [192/720 (26%)]	Loss: 12.826338
Train Epoch: 54 [256/720 (35%)]	Loss: 14.852366
Train Epoch: 54 [320/720 (43%)]	Loss: 13.885829
Train Epoch: 54 [384/720 (52%)]	Loss: 11.861229
Train Epoch: 54 [448/720 (61%)]	Loss: 12.349731
Train Epoch: 54 [512/720 (70%)]	Loss: 11.690804
Train Epoch: 54 [576/720 (78%)]	Loss: 10.249624
Train Epoch: 54 [640/720 (87%)]	Loss: 12.376845
Train Epoch: 54 [352/720 (96%)]	Loss: 14.952388
====> Epoch: 54 Average loss: 12.3818
====> Test set loss: 14.9095
Train Epoch: 55 [0/720 (0%)]	Loss: 13.409025
Train Epoch

Train Epoch: 66 [512/720 (70%)]	Loss: 13.754736
Train Epoch: 66 [576/720 (78%)]	Loss: 13.653694
Train Epoch: 66 [640/720 (87%)]	Loss: 11.814105
Train Epoch: 66 [352/720 (96%)]	Loss: 11.319656
====> Epoch: 66 Average loss: 12.2126
====> Test set loss: 14.6609
Train Epoch: 67 [0/720 (0%)]	Loss: 14.289850
Train Epoch: 67 [64/720 (9%)]	Loss: 14.299154
Train Epoch: 67 [128/720 (17%)]	Loss: 10.912335
Train Epoch: 67 [192/720 (26%)]	Loss: 10.067993
Train Epoch: 67 [256/720 (35%)]	Loss: 11.955750
Train Epoch: 67 [320/720 (43%)]	Loss: 13.194828
Train Epoch: 67 [384/720 (52%)]	Loss: 12.627112
Train Epoch: 67 [448/720 (61%)]	Loss: 13.352651
Train Epoch: 67 [512/720 (70%)]	Loss: 11.629585
Train Epoch: 67 [576/720 (78%)]	Loss: 11.163652
Train Epoch: 67 [640/720 (87%)]	Loss: 11.605281
Train Epoch: 67 [352/720 (96%)]	Loss: 10.386312
====> Epoch: 67 Average loss: 12.1619
====> Test set loss: 14.7165
Train Epoch: 68 [0/720 (0%)]	Loss: 11.863786
Train Epoch: 68 [64/720 (9%)]	Loss: 11.452944
Train Epoch:

Train Epoch: 79 [448/720 (61%)]	Loss: 11.359894
Train Epoch: 79 [512/720 (70%)]	Loss: 11.561625
Train Epoch: 79 [576/720 (78%)]	Loss: 11.171926
Train Epoch: 79 [640/720 (87%)]	Loss: 10.653967
Train Epoch: 79 [352/720 (96%)]	Loss: 10.876012
====> Epoch: 79 Average loss: 12.1315
====> Test set loss: 14.6998
Train Epoch: 80 [0/720 (0%)]	Loss: 10.618443
Train Epoch: 80 [64/720 (9%)]	Loss: 10.502734
Train Epoch: 80 [128/720 (17%)]	Loss: 11.290457
Train Epoch: 80 [192/720 (26%)]	Loss: 10.217993
Train Epoch: 80 [256/720 (35%)]	Loss: 12.794280
Train Epoch: 80 [320/720 (43%)]	Loss: 12.845371
Train Epoch: 80 [384/720 (52%)]	Loss: 12.887066
Train Epoch: 80 [448/720 (61%)]	Loss: 12.204148
Train Epoch: 80 [512/720 (70%)]	Loss: 10.039836
Train Epoch: 80 [576/720 (78%)]	Loss: 12.800296
Train Epoch: 80 [640/720 (87%)]	Loss: 12.540554
Train Epoch: 80 [352/720 (96%)]	Loss: 10.794088
====> Epoch: 80 Average loss: 12.0989
====> Test set loss: 14.6510
Train Epoch: 81 [0/720 (0%)]	Loss: 12.538062
Train Epoc

Train Epoch: 92 [448/720 (61%)]	Loss: 11.780189
Train Epoch: 92 [512/720 (70%)]	Loss: 11.075782
Train Epoch: 92 [576/720 (78%)]	Loss: 13.386102
Train Epoch: 92 [640/720 (87%)]	Loss: 11.418207
Train Epoch: 92 [352/720 (96%)]	Loss: 15.707922
====> Epoch: 92 Average loss: 12.0866
====> Test set loss: 14.6550
Train Epoch: 93 [0/720 (0%)]	Loss: 16.499165
Train Epoch: 93 [64/720 (9%)]	Loss: 10.614902
Train Epoch: 93 [128/720 (17%)]	Loss: 12.192907
Train Epoch: 93 [192/720 (26%)]	Loss: 12.888800
Train Epoch: 93 [256/720 (35%)]	Loss: 12.080653
Train Epoch: 93 [320/720 (43%)]	Loss: 10.650553
Train Epoch: 93 [384/720 (52%)]	Loss: 13.273406
Train Epoch: 93 [448/720 (61%)]	Loss: 14.092139
Train Epoch: 93 [512/720 (70%)]	Loss: 11.609072
Train Epoch: 93 [576/720 (78%)]	Loss: 12.869329
Train Epoch: 93 [640/720 (87%)]	Loss: 10.772937
Train Epoch: 93 [352/720 (96%)]	Loss: 10.159099
====> Epoch: 93 Average loss: 12.0540
====> Test set loss: 14.6352
Train Epoch: 94 [0/720 (0%)]	Loss: 10.764151
Train Epoc

In [20]:
1048576/128/128

64.0

In [21]:
model.train()
train_loss = 0
for batch_idx,( data, _) in enumerate(train_loader):
    data = data.to(device)
    data = data.reshape([-1,3, 128*128])
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    train_loss += loss.item()
    optimizer.step()
    if batch_idx % 100 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.item() / len(data)))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 3, 4, 4], but got 3-dimensional input of size [8, 3, 16384] instead